In [17]:
# Imports
from requests import get
import pandas as pd
from json import load


In [18]:
def get_api_data_and_save_to_file(api_endpoint : str, save_file_path : str, chunk_read : bool = False, chunk_size : int = 1024, append : bool = False) -> None:
    """Function makes an get request towarsd API endpoint. The response data is then saved into a file.
    
        :param str api_endpoint: API endpoint URL
        :param str save_file_path: File path of the file, where the data will be saved.
        :param bool chunk_read: If True, the data will be read and written by chunks of size determined by the chunk_size parameter. Especially useful when the data in the request is of a large size.
        :param int chunk_size: Sets the chunk size of data to be read when chunk_read is True.
        :param append: If True, the data is appended to a file instead of overwriting the data.
    
    """
    write_mode = "ab" if append == True else "wb"

    if chunk_read == True:
        with get(api_endpoint, stream = True) as response:
            with open(save_file_path, write_mode) as file:
                for chunk in response.iter_content(chunk_size):
                    file.write(chunk)
    else:
         with get(api_endpoint) as response:
            with open(save_file_path, write_mode) as file:
                    file.write(response.content)

In [19]:
get_api_data_and_save_to_file("https://662bd076de35f91de1598f6f.mockapi.io/testfidoo/companies", "og_data.json")

In [20]:
# Again, in case it"s needed we can read the json by chunks

with open("og_data.json") as f:
    data = load(f)


# flattens the address column into multiple column as it is a json sub-structure
company_df = pd.json_normalize(data)
company_df = company_df.rename(columns={"companyId":"company_id","companyName":"company_name", "address.street" : "street", "address.city" : "city", "address.zipCode" : "zip_code", "address.country" : "country"})


In [21]:
address_df = company_df[["company_id","street", "city", "zip_code", "country"]]

In [22]:
# Drop the unused columns
company_df.drop(columns=["street", "city", "zip_code", "country"], inplace=True)

In [23]:
def create_new_df_from_json_list_column(df : pd.DataFrame, columns_to_keep : list, column_to_explode : str, drop : bool = False) -> pd.DataFrame:
    """
    The entities in this JSON dataset contain lists of JSON structures, which need to be extracted into a new dataframe, this function does it. 

            :param df pd.DataFrame: Dataframe which contains the nested structure, that needs to be made into a new df.
            :param list columns_to_keep: This list contains names of columns to be transferred from the original df into the new df.
            :param str column_to_explode: Column containing the nested structure. 
            :param bool drop: If set to True, the nested column is dropped from the original dataset.
            
            :return pd.DataFrame new_df: Newly created dataframe
    """
    
    columns_to_keep.append(column_to_explode)
    new_df = df[columns_to_keep]
    
    if drop:
        df.drop(column_to_explode, inplace=True, axis=1)

    new_df = new_df.explode(column_to_explode)

    # we have to do this to not lose columns_to_keep when calling json_normalize    
    normalized_data = pd.json_normalize(new_df[column_to_explode])
    new_df.reset_index(drop=True,inplace=True)
    new_df = pd.concat([new_df.drop(columns=[column_to_explode]), normalized_data], axis=1)
    
    return new_df

In [24]:
employee_df = create_new_df_from_json_list_column(company_df, ["company_id"], "employees", drop= True)
employee_df = employee_df.rename(columns={"employeeId" : "employee_id"})

In [25]:
contacts_df = create_new_df_from_json_list_column(employee_df, ["employee_id"], "contacts", drop = True)
contacts_df = contacts_df.rename(columns={"contactType" : "contact_type", "contactId" : "contact_id", "contactValue" : "contact_value"})

In [26]:
### DROPPING DUPLICATES AND NA VALUES (most of the dataframes don't contain duplicates/NA rows, but to be uniform and as a futureproof for different instances of this dataset i perform these operations)
# Drop NA values (if contact value is lost, we don't care about the contact anymore)
contacts_df = contacts_df.dropna(subset=["contact_value"])
# Drop duplicates
contacts_df = contacts_df.drop_duplicates()
# Reset index
contacts_df = contacts_df.reset_index(drop=True)

# Drop columns if both employee_id or name are NA. We don't care about position or department in this case.
employee_df = employee_df.dropna(subset=["employee_id", "name"], thresh = 1)
# Drop duplicates
employee_df = employee_df.drop_duplicates()
#Reset index 
employee_df = employee_df.reset_index(drop=True)

# Drop addresses where street is NA
address_df = address_df.dropna(subset=["street"])
# Drop duplicates
address_df = address_df.drop_duplicates()
# Reset index 
address_df = address_df.reset_index(drop=True)

# Drop only columns that have NA in company_name
company_df = company_df.dropna(subset=["company_name"])
# Drop duplicates
company_df = company_df.drop_duplicates()
#Reset index
company_df = company_df.reset_index(drop=True)

In [27]:
# Save dataframes to CSV'
contacts_df.to_csv("contacts.csv", index=False)
employee_df.to_csv("employee.csv", index=False)
address_df.to_csv("address.csv", index = False)
company_df.to_csv("company.csv", index=False)

In [28]:
# Statistiky

# Zjistěte, kolik uživatelů celkem existuje. - předpokládám, že employee = uživatel
len(employee_df)

43

In [29]:
# Zjistěte, kolik společností je v datasetu
len(company_df)

8

In [30]:
# Pro každého uživatele zjistěte, kolik kontaktů má a jakého typu jsou tyto kontakty
contacts_df.groupby(["employee_id", "contact_type"])["contact_id"].count()


employee_id                           contact_type
010b5879-316d-4225-a0d1-ec092c820ba0  email           1
16156eb6-6772-41ed-9901-bdb4a3612ead  email           1
                                      phone           2
18690567-959f-4fbe-87d2-704bf001cdb9  email           1
                                      phone           2
1c132536-6f18-4beb-b3f4-66d994a253cf  email           2
                                      phone           1
29a3158a-a864-43e0-91cc-39b805985915  email           1
47f6bae2-65c5-4099-a491-f7b33fa76ce9  phone           3
48042062-3de3-4768-88c6-32f1a58aea89  phone           1
51d85fa8-52db-42cd-9325-71e495a9e85c  email           1
                                      phone           1
52f25ebd-6dbc-4112-b683-b607ecc1b020  email           2
                                      phone           1
5f769ef2-e5b5-466a-98b5-af139217a7fc  phone           3
6743c693-38b8-486c-aa8b-15457861810b  email           2
6d969266-9dfb-4a3d-b7d6-1f67470ded64  email          

In [31]:
employee_df.merge(contacts_df, how="left", on="employee_id")[["name", "contact_type"]].groupby(["name", "contact_type"])["contact_type"].count()

name                 contact_type
Abigail Powers       email           1
                     phone           1
Aimee Garcia         email           2
                     phone           1
Andrea Mcbride       email           2
                     phone           1
Andrew Hicks         email           1
                     phone           2
Anthony Bullock      phone           1
Brian Gardner        email           3
Bryan Lindsey        email           1
Caroline Newton      phone           3
Christopher Rogers   email           2
Connie Matthews DVM  email           2
Corey Lopez          phone           3
Cynthia Ibarra       email           2
                     phone           1
Derrick Evans        email           2
                     phone           1
Dr. Marcus Brown     email           2
                     phone           1
Erin Pearson         email           1
Jacob Thornton       email           1
                     phone           2
Jamie Hill           email    